# Imports

In [1]:
use_kaggle=False
use_jupyter=True

In [2]:
if use_jupyter:
    print("Using Jupyter")
    !dir "C:/develop/python/Final Project/Data/"
elif use_kaggle:
    print("Using Kaggle")
    !ls "/kaggle/"
else:
    print("Using Colab with drive")
    from google.colab import drive
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/Colab Notebooks/ML course/"

Using Jupyter
 Volume in drive C has no label.
 Volume Serial Number is B0A9-3BF3

 Directory of C:\develop\python\Final Project\Data

09/04/2020  13:50    <DIR>          .
09/04/2020  13:50    <DIR>          ..
09/04/2020  11:29         9,955,043 Data.zip
07/04/2020  16:24    <DIR>          extracted-masks-images
07/04/2020  16:24         1,856,537 extracted_cells.csv
07/04/2020  15:43    <DIR>          masks-images
07/04/2020  15:43    <DIR>          resized-images
               2 File(s)     11,811,580 bytes
               5 Dir(s)  446,589,669,376 bytes free


In [3]:
import numpy as np
import pandas as pd
import uuid
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import cv2
from skimage import io
from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter, ellipse

# Consts and Setups

In [4]:
if use_jupyter:
    root_input_dir = 'C:/develop/python/Final Project/Data/'
    root_output_dir = 'C:/develop/python/Final Project/Data/'
elif use_kaggle:
    root_input_dir = '/kaggle/input/'
    root_output_dir = '/kaggle/working/output/'
else:
    root_input_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'
    root_output_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'

In [5]:
cell_masks_subdirectory = 'extracted-masks-images'
minimum_cover_percent = 89
max_factor_size = 4
uuid_row_index = 1
input_csv_filename = 'extracted_cells.csv'
output_csv_filename = 'filtered_cells.csv'
images_extension = ".png"
csv_field_names = ['source_file', 'mask_uuid', 'left_x', 'right_x', 'top_y', 'bottom_y', 'id']
start_line=15001
end_line=20000

In [6]:
plt.gray()
np.seterr(divide='ignore', invalid='ignore')

# Inputs
input_csv_file_path = root_input_dir + input_csv_filename
masks_images_path = root_input_dir + cell_masks_subdirectory + '/'
input_csv_rows_list = []

# Outputs
output_csv_rows_list = []
output_csv_file_path = root_output_dir + f'{start_line}-{end_line}_' + output_csv_filename

<Figure size 432x288 with 0 Axes>

# Function Definition

In [7]:
def getMaskBestEllipse(mask_data):
    edges = canny(mask_data, sigma=2.0, low_threshold=0.55, high_threshold=0.8)
    ellipses_data = hough_ellipse(edges, accuracy=10, threshold=40, min_size=0, max_size=500)
    ellipses_data.sort(order='accumulator')
    best = list(ellipses_data[-1])
    yc, xc, a, b = [int(round(x)) for x in best[1:5]]
    orientation = best[5]
    return yc, xc, a, b, orientation

In [8]:
def resizeImage(image_data, scale_size):
    width = int(current_mask_data.shape[1] * scale_size)
    height = int(current_mask_data.shape[0] * scale_size)
    dim = (width, height)
    # resize image
    return cv2.resize(current_mask_data, dim, interpolation = cv2.INTER_AREA)

In [9]:
def getEllipseCoverPercent(mask_data, factor_size):
    # Resize the image first
    resized_image = resizeImage(mask_data, factor_size)
    mask_copy = resized_image.copy()

    # Get the best ellipse that covers the mask
    yc, xc, a, b, orientation = getMaskBestEllipse(resized_image)

    # Try to draw the best ellipse on the mask
    try:
        # Get the indices of the ellipse fill and the ellipse perimiter
        perimiter_y, perimiter_x = ellipse_perimeter(yc, xc, a, b, orientation)
        fill_y, fill_x = ellipse(yc, xc, a, b, rotation=-orientation)

        # Set 0 (black) in the mask copy where the indexes are located
        mask_copy[list(perimiter_y), list(perimiter_x)] = 0
        mask_copy[list(fill_y), list(fill_x)] = 0
        
        # Get the number of nonzero pixels - that is the number of mask pixels that the ellipse does not cover
        full_mask_nonzero = np.count_nonzero(resized_image)
        ellipse_non_zero = np.count_nonzero(mask_copy)

        return ((full_mask_nonzero - ellipse_non_zero) / full_mask_nonzero) * 100
    except:
        print(f'Could not get ellipse for factor size {factor_size}')
        return 0

# Filter Logic

In [10]:
# TODO:
# Read the csv file, which is the output of the extraction module
# Loop over the lines and foreach line extract and read the mask image (using the uuid)
# get the ellipse data from the getMaskBestEllipse function
# set the mask copy values to 0 (black) in the ellipse area
# calculate the number of nonzero pixels in the mask copy
# if it exceeds a predefined value (max_white_pixel_outside_ellipse, need to decide what is the best value) - it's not a good filter
# if it does not exceed - a good filter, insert the line from the input csv to a new output csv

with open(input_csv_file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    input_csv_rows_list = list(csv_reader)
    counter = start_line
    for line in input_csv_rows_list[start_line:end_line + 1]:
        start = time.time()
        counter = counter + 1
        # Get the mask data
        current_mask_uuid = line[uuid_row_index]
        current_mask_filename = current_mask_uuid + images_extension
        current_mask_data = io.imread(masks_images_path + current_mask_filename, as_gray=True)

        factor_size = 2
        ellipse_detected = False
        
        while factor_size <= max_factor_size and not ellipse_detected:
            # Get the coverage percent for the curretn sace factor
            current_cover_percent = getEllipseCoverPercent(current_mask_data, factor_size)

            # If the coverage percentage is greater than the wanted minimum - this is a good mask
            if current_cover_percent > minimum_cover_percent:
                ellipse_detected = True
                output_csv_rows_list.append({
                    "source_file": line[0],
                    "mask_uuid": line[1],
                    "left_x": line[2],
                    "right_x": line[3],
                    "top_y": line[4],
                    "bottom_y": line[5],
                    "id": line[6]
                })
            
            factor_size = factor_size + 1
        end = time.time()
        print(f'line number {counter} - time {end - start} seconds')
                    
                    
with open(output_csv_file_path, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=csv_field_names)
    print(f'Writing {len(output_csv_rows_list)} filtered lines to {output_csv_filename}')
    writer.writerows(output_csv_rows_list)

line number 15002 - time 1.8039531707763672 seconds
line number 15003 - time 3.5025463104248047 seconds
line number 15004 - time 7.238207578659058 seconds
line number 15005 - time 0.5338163375854492 seconds
line number 15006 - time 4.322953701019287 seconds
line number 15007 - time 0.5361557006835938 seconds
line number 15008 - time 2.2717936038970947 seconds
line number 15009 - time 31.408424139022827 seconds
line number 15010 - time 0.7899134159088135 seconds
line number 15011 - time 3.215812921524048 seconds
line number 15012 - time 0.5190026760101318 seconds
line number 15013 - time 5.289010047912598 seconds
line number 15014 - time 3.2964026927948 seconds
line number 15015 - time 3.6952741146087646 seconds
line number 15016 - time 2.5526394844055176 seconds
line number 15017 - time 2.3955397605895996 seconds
line number 15018 - time 0.5527346134185791 seconds
line number 15019 - time 2.0558674335479736 seconds
line number 15020 - time 0.5424351692199707 seconds
line number 15021 -

line number 15161 - time 0.39832329750061035 seconds
line number 15162 - time 0.4570028781890869 seconds
line number 15163 - time 3.8068642616271973 seconds
line number 15164 - time 3.901947498321533 seconds
line number 15165 - time 1.7724313735961914 seconds
line number 15166 - time 0.5579929351806641 seconds
line number 15167 - time 5.135030269622803 seconds
line number 15168 - time 0.5950777530670166 seconds
line number 15169 - time 5.625091075897217 seconds
line number 15170 - time 3.1421077251434326 seconds
line number 15171 - time 0.41773056983947754 seconds
line number 15172 - time 4.065746068954468 seconds
line number 15173 - time 3.779200553894043 seconds
line number 15174 - time 0.5363311767578125 seconds
line number 15175 - time 0.3881700038909912 seconds
line number 15176 - time 1.4242935180664062 seconds
line number 15177 - time 9.409931182861328 seconds
line number 15178 - time 12.917062282562256 seconds
line number 15179 - time 3.9977023601531982 seconds
line number 1518

line number 15320 - time 1.6711180210113525 seconds
line number 15321 - time 2.1978416442871094 seconds
line number 15322 - time 2.915586471557617 seconds
line number 15323 - time 7.4546427726745605 seconds
line number 15324 - time 3.240041494369507 seconds
line number 15325 - time 3.0644402503967285 seconds
line number 15326 - time 12.653214693069458 seconds
line number 15327 - time 4.480330467224121 seconds
line number 15328 - time 2.684234619140625 seconds
line number 15329 - time 2.4327263832092285 seconds
line number 15330 - time 2.720468044281006 seconds
line number 15331 - time 4.085559368133545 seconds
line number 15332 - time 0.5939955711364746 seconds
line number 15333 - time 0.42983078956604004 seconds
line number 15334 - time 11.781790733337402 seconds
line number 15335 - time 4.428086757659912 seconds
line number 15336 - time 4.945521354675293 seconds
line number 15337 - time 1.4233992099761963 seconds
line number 15338 - time 0.5175235271453857 seconds
line number 15339 -

line number 15477 - time 0.3904733657836914 seconds
line number 15478 - time 1.8495161533355713 seconds
line number 15479 - time 1.7277154922485352 seconds
line number 15480 - time 0.47708654403686523 seconds
line number 15481 - time 3.2874088287353516 seconds
line number 15482 - time 3.2544028759002686 seconds
line number 15483 - time 1.781951665878296 seconds
line number 15484 - time 3.5138211250305176 seconds
line number 15485 - time 0.8399016857147217 seconds
line number 15486 - time 5.270391225814819 seconds
line number 15487 - time 2.215831756591797 seconds
line number 15488 - time 0.4210214614868164 seconds
line number 15489 - time 3.004608631134033 seconds
line number 15490 - time 1.0594747066497803 seconds
line number 15491 - time 1.4618654251098633 seconds
line number 15492 - time 3.326324224472046 seconds
line number 15493 - time 3.7664270401000977 seconds
line number 15494 - time 0.4818718433380127 seconds
line number 15495 - time 0.8225545883178711 seconds
line number 1549

line number 15635 - time 9.504735469818115 seconds
line number 15636 - time 3.394958019256592 seconds
line number 15637 - time 0.43622851371765137 seconds
line number 15638 - time 0.5009732246398926 seconds
line number 15639 - time 5.862928628921509 seconds
line number 15640 - time 5.87839937210083 seconds
line number 15641 - time 2.0008652210235596 seconds
line number 15642 - time 2.6020021438598633 seconds
line number 15643 - time 7.842094421386719 seconds
line number 15644 - time 1.6565182209014893 seconds
line number 15645 - time 2.877581834793091 seconds
line number 15646 - time 0.46291017532348633 seconds
line number 15647 - time 4.418987274169922 seconds
line number 15648 - time 4.10960578918457 seconds
line number 15649 - time 15.672232866287231 seconds
line number 15650 - time 3.3714311122894287 seconds
line number 15651 - time 4.112738132476807 seconds
line number 15652 - time 0.5978891849517822 seconds
line number 15653 - time 2.754281520843506 seconds
line number 15654 - ti

...

line number 19262 - time 4.479744911193848 seconds
line number 19263 - time 12.546657800674438 seconds
line number 19264 - time 0.4788649082183838 seconds
line number 19265 - time 2.8561480045318604 seconds
line number 19266 - time 3.64829421043396 seconds
line number 19267 - time 0.47814440727233887 seconds
line number 19268 - time 7.179698944091797 seconds
line number 19269 - time 2.5171284675598145 seconds
line number 19270 - time 2.8565261363983154 seconds
line number 19271 - time 17.56662082672119 seconds
line number 19272 - time 2.1669368743896484 seconds
line number 19273 - time 2.2350616455078125 seconds
line number 19274 - time 4.828077077865601 seconds
line number 19275 - time 3.581404447555542 seconds
line number 19276 - time 2.173011064529419 seconds
line number 19277 - time 3.3802073001861572 seconds
line number 19278 - time 1.982067346572876 seconds
line number 19279 - time 2.0676329135894775 seconds
line number 19280 - time 2.142659902572632 seconds
line number 19281 - t

line number 19419 - time 3.329617977142334 seconds
line number 19420 - time 1.6218948364257812 seconds
line number 19421 - time 0.5287911891937256 seconds
line number 19422 - time 6.3565239906311035 seconds
line number 19423 - time 2.794720411300659 seconds
line number 19424 - time 1.5378139019012451 seconds
line number 19425 - time 3.6225695610046387 seconds
Could not get ellipse for factor size 4
line number 19426 - time 7.080009937286377 seconds
line number 19427 - time 0.37709784507751465 seconds
line number 19428 - time 1.5081052780151367 seconds
line number 19429 - time 2.3964831829071045 seconds
line number 19430 - time 2.33746075630188 seconds
line number 19431 - time 2.1612329483032227 seconds
line number 19432 - time 1.00370454788208 seconds
line number 19433 - time 0.5536909103393555 seconds
line number 19434 - time 8.781099319458008 seconds
line number 19435 - time 4.476028919219971 seconds
line number 19436 - time 6.076786041259766 seconds
line number 19437 - time 2.185425

Could not get ellipse for factor size 2
line number 19576 - time 1.6981375217437744 seconds
line number 19577 - time 2.438664197921753 seconds
line number 19578 - time 0.5478484630584717 seconds
line number 19579 - time 2.3871562480926514 seconds
line number 19580 - time 0.3730025291442871 seconds
line number 19581 - time 1.6002781391143799 seconds
line number 19582 - time 1.117401123046875 seconds
Could not get ellipse for factor size 3
line number 19583 - time 19.30273723602295 seconds
line number 19584 - time 3.827239990234375 seconds
line number 19585 - time 0.4379699230194092 seconds
line number 19586 - time 1.7210566997528076 seconds
line number 19587 - time 4.819447755813599 seconds
line number 19588 - time 1.7429571151733398 seconds
line number 19589 - time 3.2294020652770996 seconds
line number 19590 - time 3.5324645042419434 seconds
line number 19591 - time 2.5023558139801025 seconds
line number 19592 - time 10.324547529220581 seconds
line number 19593 - time 3.17003822326660

line number 19732 - time 3.694380760192871 seconds
line number 19733 - time 0.550398588180542 seconds
line number 19734 - time 3.3106565475463867 seconds
line number 19735 - time 2.3972408771514893 seconds
line number 19736 - time 2.8707375526428223 seconds
line number 19737 - time 4.6132237911224365 seconds
line number 19738 - time 10.69472336769104 seconds
line number 19739 - time 0.48502612113952637 seconds
line number 19740 - time 3.560462713241577 seconds
line number 19741 - time 4.86506986618042 seconds
line number 19742 - time 0.47765302658081055 seconds
line number 19743 - time 0.5644431114196777 seconds
line number 19744 - time 7.463775634765625 seconds
line number 19745 - time 3.7624192237854004 seconds
line number 19746 - time 4.894151449203491 seconds
line number 19747 - time 0.4938936233520508 seconds
line number 19748 - time 0.5055906772613525 seconds
line number 19749 - time 3.6745190620422363 seconds
line number 19750 - time 0.6705033779144287 seconds
line number 19751 

line number 19886 - time 0.9208176136016846 seconds
line number 19887 - time 3.225996255874634 seconds
line number 19888 - time 1.9841861724853516 seconds
line number 19889 - time 0.37778234481811523 seconds
line number 19890 - time 3.893927574157715 seconds
line number 19891 - time 7.1748034954071045 seconds
line number 19892 - time 3.921680450439453 seconds
line number 19893 - time 0.43507814407348633 seconds
line number 19894 - time 1.1701066493988037 seconds
line number 19895 - time 2.5959644317626953 seconds
line number 19896 - time 0.5040898323059082 seconds
line number 19897 - time 1.945044755935669 seconds
line number 19898 - time 0.621537446975708 seconds
line number 19899 - time 3.121264696121216 seconds
line number 19900 - time 2.480423927307129 seconds
line number 19901 - time 1.5317025184631348 seconds
line number 19902 - time 2.0165648460388184 seconds
line number 19903 - time 3.9865097999572754 seconds
line number 19904 - time 1.886216402053833 seconds
line number 19905 